# Frozen-in Magnetic Flux: Circular Advection Benchmark

**Validation case**: Circular advection of a magnetic loop in the ideal MHD limit

## Learning Objectives

After completing this notebook, you will understand:

1. The magnetic induction equation and its ideal-MHD limit ($R_m \gg 1$)
2. Alfvén's frozen flux theorem: magnetic field lines move with the plasma
3. The circular advection benchmark for validating MHD solvers
4. How numerical diffusion affects advection-dominated simulations

---
## 1. Physics Background

### Magnetic Induction Equation

The evolution of the magnetic field $\mathbf{B}$ is governed by:

$$\frac{\partial \mathbf{B}}{\partial t} = \nabla \times (\mathbf{v} \times \mathbf{B}) + \eta \nabla^2 \mathbf{B}$$

where $\eta$ is the magnetic diffusivity.

### Ideal MHD Limit ($R_m \gg 1$)

When the magnetic Reynolds number $R_m = vL/\eta \gg 1$, diffusion is negligible:

$$\frac{\partial \mathbf{B}}{\partial t} = \nabla \times (\mathbf{v} \times \mathbf{B})$$

This implies **Alfvén's frozen flux theorem**: magnetic field lines are "frozen" into the plasma and move with it.

### Circular Advection Benchmark

A localized magnetic loop is advected by rigid body rotation:
- Velocity field: $v_x = -\omega y$, $v_y = \omega x$
- After one period $T = 2\pi/\omega$, the loop should return to its initial position

This tests:
1. **Numerical diffusion** (amplitude preservation)
2. **Dispersion** (shape preservation)
3. **div(B) = 0** constraint maintenance

---
## 2. Configuration Setup

In [ ]:
# Standard imports
import jax
import jax.numpy as jnp
import jax.lax as lax
import numpy as np
import matplotlib.pyplot as plt

# jax-frc imports
from jax_frc.configurations import FrozenFluxConfiguration
from jax_frc.solvers.explicit import RK4Solver
from jax_frc.validation.metrics import l2_error

# Plotting style
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

In [ ]:
# === CONFIGURATION ===
config = FrozenFluxConfiguration(
    nx=64, ny=64, nz=1,      # Pseudo-2D in x-y plane
    domain_extent=1.0,       # x,y in [-1, 1]
    loop_x0=0.5,             # Loop center
    loop_y0=0.0,
    loop_radius=0.3,         # Loop radius
    loop_amplitude=1.0,      # Vector potential amplitude
    eta=1e-8,                # Very small resistivity (Rm >> 1)
)

runtime = config.default_runtime()
t_end = runtime['t_end']
dt = runtime['dt']

print(f"Grid: {config.nx} x {config.ny} x {config.nz}")
print(f"Domain: x,y in [-{config.domain_extent}, {config.domain_extent}]")
print(f"Loop center: ({config.loop_x0}, {config.loop_y0}), radius: {config.loop_radius}")
print(f"Angular velocity: omega = {config.omega:.4f} rad/s")
print(f"Rotation period: T = {config.rotation_period():.4f} s")
print(f"Simulation time: t_end = {t_end:.4f} s (quarter rotation)")
print(f"Timestep: dt = {dt:.6f} s")
print(f"Magnetic Reynolds number: Rm = {config.magnetic_reynolds_number():.2e}")

In [ ]:
# Build simulation components
geometry = config.build_geometry()
state0 = config.build_initial_state(geometry)
model = config.build_model()
solver = RK4Solver()

# Extract 2D slice at z=0
z_idx = geometry.nz // 2
x = np.array(geometry.x_grid[:, :, z_idx])
y = np.array(geometry.y_grid[:, :, z_idx])

# Initial B magnitude
B_mag_init = np.sqrt(np.sum(np.array(state0.B[:, :, z_idx, :])**2, axis=-1))

# Plot initial condition
fig, ax = plt.subplots(figsize=(6, 5))
im = ax.contourf(x, y, B_mag_init, levels=20, cmap='viridis')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title('Initial |B| (magnetic loop)')
ax.set_aspect('equal')
plt.colorbar(im, ax=ax, label='|B| [T]')

# Mark loop center
ax.plot(config.loop_x0, config.loop_y0, 'r+', markersize=10, markeredgewidth=2)
circle = plt.Circle((config.loop_x0, config.loop_y0), config.loop_radius, 
                     fill=False, color='red', linestyle='--')
ax.add_patch(circle)
plt.show()

---
## 3. Run Simulation

We evolve the magnetic loop under rigid body rotation. The analytic solution is the initial field rotated by angle $\theta = \omega t$.

In [ ]:
n_steps = int(t_end / dt)
print(f"Running {n_steps} steps...")

@jax.jit
def scan_step(state, _):
    new_state = solver.step(state, dt, model, geometry)
    return new_state, None

final_state, _ = lax.scan(scan_step, state0, None, length=n_steps)
print("Done!")

In [ ]:
# Get analytic solution at t_end
B_analytic = config.analytic_solution(geometry, t_end)

# Extract 2D slices
B_mag_final = np.sqrt(np.sum(np.array(final_state.B[:, :, z_idx, :])**2, axis=-1))
B_mag_analytic = np.sqrt(np.sum(np.array(B_analytic[:, :, z_idx, :])**2, axis=-1))

# Compute metrics
l2_err = float(l2_error(jnp.asarray(final_state.B), jnp.asarray(B_analytic)))
peak_init = float(jnp.max(jnp.sqrt(jnp.sum(state0.B**2, axis=-1))))
peak_final = float(jnp.max(jnp.sqrt(jnp.sum(final_state.B**2, axis=-1))))
peak_ratio = peak_final / peak_init

print(f"L2 error vs analytic: {l2_err:.4f}")
print(f"Peak amplitude ratio: {peak_ratio:.4f}")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Analytic solution
im0 = axes[0].contourf(x, y, B_mag_analytic, levels=20, cmap='viridis')
axes[0].set_title(f'Analytic |B| (t={t_end:.2f}s)')
axes[0].set_xlabel('x [m]')
axes[0].set_ylabel('y [m]')
axes[0].set_aspect('equal')
plt.colorbar(im0, ax=axes[0], label='|B| [T]')

# Numerical solution
im1 = axes[1].contourf(x, y, B_mag_final, levels=20, cmap='viridis')
axes[1].set_title(f'Numerical |B| (t={t_end:.2f}s)')
axes[1].set_xlabel('x [m]')
axes[1].set_ylabel('y [m]')
axes[1].set_aspect('equal')
plt.colorbar(im1, ax=axes[1], label='|B| [T]')

# Error
error = B_mag_final - B_mag_analytic
im2 = axes[2].contourf(x, y, error, levels=20, cmap='RdBu_r')
axes[2].set_title('Error (Numerical - Analytic)')
axes[2].set_xlabel('x [m]')
axes[2].set_ylabel('y [m]')
axes[2].set_aspect('equal')
plt.colorbar(im2, ax=axes[2], label='Error [T]')

plt.tight_layout()
plt.show()

---
## 4. Discussion

### Numerical Diffusion

The central difference scheme used for advection introduces numerical diffusion, which causes:
- Amplitude decay (peak |B| decreases over time)
- Spreading of the magnetic loop

This is a known limitation of central difference advection schemes. Higher-order schemes (e.g., WENO, upwind) would reduce this diffusion.

### Validation Criteria

For this benchmark with central differences:
- L2 error < 35% (quarter rotation)
- Peak amplitude ratio > 75%

These thresholds account for the inherent numerical diffusion while still validating that the solver correctly advects the magnetic field.

### Try Next

- Increase grid resolution to reduce numerical diffusion
- Compare with the **magnetic diffusion** notebook for the opposite limit ($R_m \ll 1$)
- Run for a full rotation to see cumulative diffusion effects